In [1]:
!pip install -q tensorflow-recommenders

     |████████████████████████████████| 89 kB 4.0 MB/s 


In [2]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs
import pandas as pd
import sys
from datetime import datetime

In [4]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
if 'google.colab' in str(get_ipython()):

  # To run in Gogogle Colab

  #@title Mount your Google Drive

  %matplotlib inline
  %load_ext autoreload
  %autoreload 2

  from google.colab import drive
  #drive.mount('/content/gdrive')
  drive.mount("/content/gdrive", force_remount=True)

  folder = "/content/gdrive/MyDrive/COMP585" #@param {type:"string"}
  !ln -Ts $folder /content/IFT6759 2> /dev/null

  # Add the assignment folder to Python path
  if '/content/COMP585' not in sys.path:
      sys.path.insert(0, '/content/COMP585')

Mounted at /content/gdrive


# Retrieval Model

## Data

In [29]:
if 'google.colab' in str(get_ipython()):
  watched = pd.read_csv('/content/gdrive/MyDrive/COMP585/data_watched.csv', sep=',')
  ratings = pd.read_csv('/content/gdrive/MyDrive/COMP585/data.csv', sep=',')
  movies = pd.read_csv('/content/gdrive/MyDrive/COMP585/data_movie.csv', sep=',')
  users = pd.read_csv('/content/gdrive/MyDrive/COMP585/data_user.csv', sep=';')
else:
  watched = pd.read_csv('data_watched.csv', sep=',')
  ratings = pd.read_csv('data.csv', sep=',')
  movies = pd.read_csv('data_movie.csv', sep=',')
  users = pd.read_csv('data_user.csv', sep=';')

#Data Preprocess

In [30]:
use_ratings_retrieval = True

In [31]:
# movies

# length binned:
bins = [0,30, 90, 120, 200,1500]
labels = [0,1,2,3,4]
movies['length_binned'] = pd.cut(movies['length'], bins=bins, labels=labels)

# year binned
movies['release_date'] = movies['release_date'].fillna('2000-01-01') # fill na !!!!
movies['original_language'] = movies['original_language'].fillna("en").astype(str) # fill na !!!!


movies['release_date'] = movies['release_date'].apply(lambda row:  datetime.strptime(str(row),'%Y-%m-%d').year ) 

bins = [1850,1920,1970,1990,2010,2020,2025]
labels = [0,1,2,3,4,5]
movies['year_binned'] = pd.cut(movies['release_date'], bins=bins, labels=labels)

movies['genres'] = movies.apply(lambda row : eval(row['genres']), axis = 1)
movies['genres'] = movies['genres'].apply(lambda row : [d['name'] for d in row if 'id' in d])

unique_genres = movies["genres"].explode().unique() 

movies = pd.concat([
        movies[['movieID','length_binned', 'year_binned', 'vote_average', 'popularity', 'original_language']],
        movies.genres.apply(lambda x: pd.Series(1, x)).fillna(0)
    ], axis=1)

In [32]:
movies

,movieID,length_binned,year_binned,vote_average,popularity,original_language,Comedy,Mystery,Crime,Drama,...,Adventure,War,Horror,Western,Fantasy,Family,History,TV Movie,Music,Foreign
0,++++++1959,2,1,6.5,0.142537,el,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,++++2013,1,4,10.0,0.070407,fa,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,+la+mode+1993,1,3,0.0,0.000082,en,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,+laventure+2008,2,3,5.0,3.282339,fr,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,+nos+amours+1983,2,2,5.6,3.609326,fr,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25502,zulu+1964,3,1,7.4,11.604242,en,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25503,zulu+2013,2,4,6.7,6.826661,en,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25504,zulu+dawn+1979,2,2,5.6,2.271555,en,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25505,zus++zo+2001,2,3,6.3,0.712200,nl,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
N_movies = movies.shape[0]

In [39]:
users

,userID,age,occupation,gender
0,1,34,sales/marketing,M
1,2,33,college/grad student,M
2,3,29,scientist,M
3,4,30,other or not specified,M
4,5,26,scientist,M
...,...,...,...,...
999995,999996,31,scientist,M
999996,999997,27,scientist,M
999997,999998,37,executive/managerial,M
999998,999999,29,scientist,M


In [40]:
#users

bins = [0,18,25, 35, 45, 55, 65,100]
labels = [0,1,2,3,4,5,6]
users['age_binned'] = pd.cut(users['age'], bins=bins, labels=labels)

users = users[['userID', 'age','gender', 'age_binned', 'occupation']]
users['userID'] = users['userID'].astype(str)

In [41]:
users

,userID,age,gender,age_binned,occupation
0,1,34,M,2,sales/marketing
1,2,33,M,2,college/grad student
2,3,29,M,2,scientist
3,4,30,M,2,other or not specified
4,5,26,M,2,scientist
...,...,...,...,...,...
999995,999996,31,M,2,scientist
999996,999997,27,M,2,scientist
999997,999998,37,M,3,executive/managerial
999998,999999,29,M,2,scientist


In [42]:
# watched

# adjust type of the column to the model
watched['userID'] = watched['userID'].astype(str)
watched

,userID,movieID,watchTime
0,177165,geronimo+an+american+legend+1993,98
1,337283,men+with+guns+1998,48
2,464512,from+russia+with+love+1963,99
3,63420,iron+man+2008,92
4,479764,deception+1946,71
...,...,...,...
9999995,216582,the+dark+tower+1943,72
9999996,276532,2001+a+space+odyssey+1968,43
9999997,239797,the+hunger+games+mockingjay+-+part+1+2014,59
9999998,264306,bambi+1942,49


In [43]:
if use_ratings_retrieval:
  ratings['userID'] = ratings['userID'].astype(str)
  ratings = ratings.loc[ratings['ratings']>3]
  ratings = ratings[['userID', 'movieID']]
  watched = watched[['userID', 'movieID']]
  watched = pd.concat([watched, ratings], ignore_index=True)
  watched

In [44]:
watched = watched.merge(movies, left_on='movieID', right_on='movieID', how='left')
watched = watched.merge(users, left_on='userID', right_on='userID', how='left')

In [45]:
watched.isna().sum()

userID                    0
movieID                   0
length_binned        119703
year_binned          118757
vote_average         118757
popularity           118757
original_language    118757
Comedy               118757
Mystery              118757
Crime                118757
Drama                118757
Romance              118757
Documentary          118757
Thriller             118757
Action               118757
Animation            118757
Science Fiction      118757
Adventure            118757
War                  118757
Horror               118757
Western              118757
Fantasy              118757
Family               118757
History              118757
TV Movie             118757
Music                118757
Foreign              118757
age                       0
gender                    0
age_binned                0
occupation                0
dtype: int64

In [ ]:
watched['length_binned'] = watched['length_binned'].fillna(2) ##########################
watched['year_binned'] = watched['year_binned'].fillna(2) ##########################
watched['original_language'] = watched['original_language'].fillna('en') ##########################
watched = watched.drop(['popularity','vote_average'], axis=1)
watched = watched.fillna(0)
watched

In [ ]:
watched = watched.drop_duplicates()
watched

In [ ]:
movies = movies.loc[movies['vote_average']>6]
movies = movies.loc[movies['popularity']>1]
movies

## Get the train and test size

In [ ]:
N = watched.shape[0]
N_train = N*80//100
N_test = N - N_train

N_train, N_test

## Create Tensorflow Datasets

In [ ]:
watched = tf.data.Dataset.from_tensor_slices(dict(watched))
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

In [ ]:
watched = watched.map(lambda x: {
    "movie_title": x["movieID"],
    "movie_length_binned": x['length_binned'],	
    "movie_year_binned": x['year_binned'],
    "movie_language": x["original_language"],
    "user_id": x["userID"],
    "user_gender": x["gender"],
    "user_age_binned": x["age_binned"],
    "user_occupation": x["occupation"],
    "Comedy" : x["Comedy"],
    "Mystery": x["Mystery"],            
    "Crime": x["Crime"],              
    "Drama": x["Drama"],              
    "Romance": x["Romance"],            
    "Documentary": x["Documentary"],        
    "Thriller": x["Thriller"],           
    "Action": x["Action"],             
    "Animation": x["Animation"],          
    "Science Fiction": x["Science Fiction"],    
    "Adventure": x["Adventure"],          
    "War": x["War"],               
    "Horror": x["Horror"],             
    "Western": x["Western"],            
    "Fantasy": x["Fantasy"],            
    "Family": x["Family"],             
    "History": x["History"],            
    "TV Movie": x["TV Movie"],           
    "Music": x["Music"],              
    "Foreign": x["Foreign"],            
    })



movies = movies.map(lambda x: {
    "candidate_title": x["movieID"],
    "candidate_length_binned": x['length_binned'],	
    "candidate_year_binned": x['year_binned'],
    "candidate_language": x["original_language"],
    "candidate_Comedy" : x["Comedy"],
    "candidate_Mystery": x["Mystery"],            
    "candidate_Crime": x["Crime"],              
    "candidate_Drama": x["Drama"],              
    "candidate_Romance": x["Romance"],            
    "candidate_Documentary": x["Documentary"],        
    "candidate_Thriller": x["Thriller"],           
    "candidate_Action": x["Action"],             
    "candidate_Animation": x["Animation"],          
    "candidate_Science_Fiction": x["Science Fiction"],    
    "candidate_Adventure": x["Adventure"],          
    "candidate_War": x["War"],               
    "candidate_Horror": x["Horror"],             
    "candidate_Western": x["Western"],            
    "candidate_Fantasy": x["Fantasy"],            
    "candidate_Family": x["Family"],             
    "candidate_History": x["History"],            
    "candidate_TV_Movie": x["TV Movie"],           
    "candidate_Music": x["Music"],              
    "candidate_Foreign": x["Foreign"],  
    })

  


In [ ]:
for x in watched.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'Action': 1.0,
 'Adventure': 0.0,
 'Animation': 0.0,
 'Comedy': 0.0,
 'Crime': 0.0,
 'Documentary': 0.0,
 'Drama': 1.0,
 'Family': 0.0,
 'Fantasy': 0.0,
 'Foreign': 0.0,
 'History': 1.0,
 'Horror': 0.0,
 'Music': 0.0,
 'Mystery': 0.0,
 'Romance': 0.0,
 'Science Fiction': 0.0,
 'TV Movie': 0.0,
 'Thriller': 0.0,
 'War': 0.0,
 'Western': 1.0,
 'movie_language': b'en',
 'movie_length_binned': 2,
 'movie_title': b'geronimo+an+american+legend+1993',
 'movie_year_binned': 3,
 'user_age_binned': 2,
 'user_gender': b'M',
 'user_id': b'177165',
 'user_occupation': b'college/grad student'}


In [ ]:
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'candidate_Action': 0.0,
 'candidate_Adventure': 0.0,
 'candidate_Animation': 0.0,
 'candidate_Comedy': 1.0,
 'candidate_Crime': 0.0,
 'candidate_Documentary': 0.0,
 'candidate_Drama': 0.0,
 'candidate_Family': 0.0,
 'candidate_Fantasy': 0.0,
 'candidate_Foreign': 0.0,
 'candidate_History': 0.0,
 'candidate_Horror': 0.0,
 'candidate_Music': 0.0,
 'candidate_Mystery': 0.0,
 'candidate_Romance': 0.0,
 'candidate_Science_Fiction': 0.0,
 'candidate_TV_Movie': 0.0,
 'candidate_Thriller': 0.0,
 'candidate_War': 0.0,
 'candidate_Western': 0.0,
 'candidate_language': b'fr',
 'candidate_length_binned': 2.0,
 'candidate_title': b'+nous+la+libert+1931',
 'candidate_year_binned': 1}


## Random split (80%-20%) in the test set). Should we consider time to split ?



In [ ]:
tf.random.set_seed(42)
shuffled = watched.shuffle(N, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(N_train)
test = shuffled.skip(N_train).take(N_test)

#train = watched.take(N_train)
#test = watched.skip(N_train).take(N_test)

cached_train = train.shuffle(N_train).batch(8192).cache()
cached_test = test.batch(8192).cache()

## Unique values to be used as diccionary in StringLookup and IntegerLookup

In [ ]:
movie_titles = movies.batch(N_movies).map(lambda x: x["candidate_title"])
years_binned = movies.batch(N_movies).map(lambda x: x["candidate_year_binned"])
lengths_binned = movies.batch(N_movies).map(lambda x: x["candidate_length_binned"])
languages = movies.batch(N_movies).map(lambda x: x["candidate_language"]) 
occupations = watched.batch(N).map(lambda x: x["user_occupation"])
user_ids = watched.batch(N).map(lambda x: x["user_id"])
user_genders = watched.batch(N).map(lambda x: x["user_gender"])
user_ages_binned = watched.batch(N).map(lambda x: x["user_age_binned"])



unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_years_binned = np.unique(np.concatenate(list(years_binned)))
unique_lengths_binned = np.unique(np.concatenate(list(lengths_binned)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
unique_user_genders = np.unique(np.concatenate(list(user_genders)))
unique_user_ages_binned = np.unique(np.concatenate(list(user_ages_binned)))
unique_languages = np.unique(np.concatenate(list(languages)))
unique_occupations = np.unique(np.concatenate(list(occupations)))


In [28]:
len(unique_movie_titles),len(unique_years_binned),len(unique_lengths_binned),len(unique_user_ids),len(unique_user_genders),len(unique_user_ages_binned),len(unique_languages),len(unique_occupations)

(10104, 5, 6, 189497, 2, 7, 57, 21)

In [ ]:
unique_movie_titles[:10]

array([b'+nous+la+libert+1931', b'...and+god+created+woman+1956',
       b'...and+justice+for+all+1979', b'10+items+or+less+2006',
       b'10+rillington+place+1971', b'10+things+i+hate+about+you+1999',
       b'10+to+midnight+1983', b'100+bloody+acres+2012',
       b'101+reykjavik+2000', b'102+minutes+that+changed+america+2008'],
      dtype=object)

In [ ]:
unique_user_ids[:10]

array([b'1', b'10', b'10000', b'100001', b'100003', b'100004', b'100006',
       b'10001', b'100010', b'100011'], dtype=object)

##  Model (Retrieval)

In [ ]:
embedding_dimension = 32

class RetrievalUserModel(tf.keras.Model):
  def __init__(self, layer_sizes):
    super().__init__()
    
    self.user_embedding = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32)
    ])
    
    self.age_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_user_ages_binned, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ages_binned) + 1, 4),
    ])
    
    self.gender_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_genders, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_genders) + 1, 1),
    ])

    self.occupation_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_occupations, mask_token=None),
        tf.keras.layers.Embedding(len(unique_occupations) + 1, 8),
    ])
    

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      #self.dense_layers.add(tf.keras.layers.Dropout(0.2, (layer_size,)))
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, kernel_regularizer='l2', activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))
 
  def call(self, inputs): 


    feature_embedding = tf.concat(values=[self.user_embedding(inputs["user_id"]),
                                          self.age_embedding(inputs["user_age_binned"]),
                                          self.gender_embedding(inputs["user_gender"]),
                                          self.occupation_embedding(inputs["user_occupation"])], axis=1)
    
    return self.dense_layers(feature_embedding )


class RetrievalMovieModel(tf.keras.Model):
  def __init__(self, layer_sizes):
    
    super().__init__()
    
    
    self.title_embedding = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)])
    
    self.lenght_embedding = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(
      vocabulary=unique_lengths_binned, mask_token=None),
    tf.keras.layers.Embedding(len(unique_lengths_binned) + 1, 2)])

    self.year_embedding = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(
      vocabulary=unique_years_binned, mask_token=None),
    tf.keras.layers.Embedding(len(unique_years_binned) + 1, 2)])

    self.language_embedding = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_languages, mask_token=None),
    tf.keras.layers.Embedding(len(unique_languages) + 1, 8)])

    self.genre_embedding = tf.keras.layers.Dense(4, activation="relu")


    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      #self.dense_layers.add(tf.keras.layers.Dropout(0.2, (layer_size,)))
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, kernel_regularizer='l2', activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))
    
  
  def call(self, inputs):
    
    
    Comedy =  tf.expand_dims(inputs["candidate_Comedy"], axis=1)
    Mystery =  tf.expand_dims(inputs["candidate_Mystery"], axis=1)
    Crime  =  tf.expand_dims(inputs["candidate_Crime"], axis=1)
    Drama =  tf.expand_dims(inputs["candidate_Drama"], axis=1)
    Romance  =  tf.expand_dims(inputs["candidate_Romance"], axis=1)
    Documentary =  tf.expand_dims(inputs["candidate_Documentary"], axis=1)
    Thriller  =  tf.expand_dims(inputs["candidate_Thriller"], axis=1)
    Action =  tf.expand_dims(inputs["candidate_Action"], axis=1)
    Animation =  tf.expand_dims(inputs["candidate_Animation"], axis=1)
    Science_Fiction =  tf.expand_dims(inputs["candidate_Science_Fiction"], axis=1)
    Adventure =  tf.expand_dims(inputs["candidate_Adventure"], axis=1)
    War  =  tf.expand_dims(inputs["candidate_War"], axis=1)
    Horror  =  tf.expand_dims(inputs["candidate_Horror"], axis=1)
    Western =  tf.expand_dims(inputs["candidate_Western"], axis=1)
    Fantasy =  tf.expand_dims(inputs["candidate_Fantasy"], axis=1)
    Family =  tf.expand_dims(inputs["candidate_Family"], axis=1)
    History =  tf.expand_dims(inputs["candidate_History"], axis=1)
    TV_Movie =  tf.expand_dims(inputs["candidate_TV_Movie"], axis=1)
    Music =  tf.expand_dims(inputs["candidate_Music"], axis=1)
    Foreign  =  tf.expand_dims(inputs["candidate_Foreign"], axis=1)

    genres = tf.concat(values=[Comedy,Mystery,Crime,Drama,Romance,Documentary,Thriller,Action,
                               Animation,Science_Fiction,Adventure,War,Horror,Western,
                               Fantasy,Family,History,TV_Movie,Music,Foreign],axis=1)
        
    
    feature_embedding = tf.concat(values=[self.title_embedding(inputs['candidate_title']),self.lenght_embedding(inputs['candidate_length_binned']),
                                          self.year_embedding(inputs['candidate_year_binned']),self.genre_embedding(genres), 
                                          self.language_embedding(inputs['candidate_language'])], axis=1)
    
    #feature_embedding = tf.concat(values=[self.title_embedding(inputs['candidate_title']),self.lenght_embedding(inputs['candidate_length_binned']),
    #                                      self.year_embedding(inputs['candidate_year_binned']),genres, axis=1)

    #feature_embedding = tf.concat(values=[self.title_embedding(inputs['candidate_title']),self.lenght_embedding(inputs['candidate_length_binned']),
    #                                      self.year_embedding(inputs['candidate_year_binned'])], axis=1)

    return self.dense_layers(feature_embedding )

class RecRetModel(tfrs.Model):
  
  def __init__(self, layer_sizes):
    super().__init__()
    
    
    self.query_model = tf.keras.Sequential([
      RetrievalUserModel(layer_sizes),
      tf.keras.layers.Dense(32)
    ])
    
    self.candidate_model = tf.keras.Sequential([
      RetrievalMovieModel(layer_sizes),
      tf.keras.layers.Dense(32)
    ])
    
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(30000).map(self.candidate_model),
            ks = (5,20,500)
        ),
    )
    
  #def compute_loss(self, features : Dict[Text, tf.Tensor], training=False):
  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    

    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "user_age_binned": features["user_age_binned"],
        "user_gender": features["user_gender"],
        "user_occupation": features["user_occupation"], 
        "movie_length_binned": features['movie_length_binned'],	
        "movie_year_binned": features['movie_year_binned'],
        "movie_language": features['movie_language'],
    })


    
    movie_embeddings = self.candidate_model({
        "candidate_title": features["movie_title"],
        "candidate_length_binned": features['movie_length_binned'],	
        "candidate_year_binned": features['movie_year_binned'],
        "candidate_language": features['movie_language'],
        "candidate_Comedy" : features["Comedy"],
        "candidate_Mystery": features["Mystery"],            
        "candidate_Crime": features["Crime"],              
        "candidate_Drama": features["Drama"],              
        "candidate_Romance": features["Romance"],            
        "candidate_Documentary": features["Documentary"],        
        "candidate_Thriller": features["Thriller"],           
        "candidate_Action": features["Action"],             
        "candidate_Animation": features["Animation"],          
        "candidate_Science_Fiction": features["Science Fiction"],    
        "candidate_Adventure": features["Adventure"],          
        "candidate_War": features["War"],               
        "candidate_Horror": features["Horror"],             
        "candidate_Western": features["Western"],            
        "candidate_Fantasy": features["Fantasy"],            
        "candidate_Family": features["Family"],             
        "candidate_History": features["History"],            
        "candidate_TV_Movie": features["TV Movie"],           
        "candidate_Music": features["Music"],              
        "candidate_Foreign": features["Foreign"],    

    })
    
    
    return self.task(query_embeddings, movie_embeddings)


## Train, Evaluate and Save the Model

In [ ]:
model_1 = RecRetModel([32])
model_1.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.2))
model_1.fit(cached_train, epochs=50, validation_data=cached_test)
#model_1.fit(cached_train, epochs=50)

Epoch 1/50


26/26 [==============================] - ETA: 0s - factorized_top_k/top_5_categorical_accuracy: 0.0217 - factorized_top_k/top_20_categorical_accuracy: 0.0251 - factorized_top_k/top_500_categorical_accuracy: 0.0590 - loss: 80109.6626 - regularization_loss: 0.0000e+00 - total_loss: 80109.6626

26/26 [==============================] - 103s 4s/step - factorized_top_k/top_5_categorical_accuracy: 0.0217 - factorized_top_k/top_20_categorical_accuracy: 0.0251 - factorized_top_k/top_500_categorical_accuracy: 0.0590 - loss: 78146.1548 - regularization_loss: 0.0000e+00 - total_loss: 78146.1548 - val_factorized_top_k/top_5_categorical_accuracy: 1.5374e-04 - val_factorized_top_k/top_20_categorical_accuracy: 7.6871e-04 - val_factorized_top_k/top_500_categorical_accuracy: 0.0229 - val_loss: 22985.9531 - val_regularization_loss: 0.0000e+00 - val_total_loss: 22985.9531
Epoch 2/50
26/26 [==============================] - 80s 3s/step - factorized_top_k/top_5_categorical_accuracy: 0.0079 - factorized_top_k/top_20_categorical_accuracy: 0.0271 - factorized_top_k/top_500_categorical_accuracy: 0.2339 - loss: 67045.7459 - regularization_loss: 0.0000e+00 - total_loss: 67045.7459 - val_factorized_top_k/top_5_categorical_accuracy: 4.6123e-04 - val_factorized_top_k/top_20_categorical_accuracy: 0.0021 

In [ ]:
model_1.evaluate(cached_test, return_dict=True)

7/7 [==============================] - 15s 2s/step - factorized_top_k/top_5_categorical_accuracy: 0.1237 - factorized_top_k/top_20_categorical_accuracy: 0.1445 - factorized_top_k/top_500_categorical_accuracy: 0.3279 - loss: 177095.1914 - regularization_loss: 0.0000e+00 - total_loss: 177095.1914


{'factorized_top_k/top_5_categorical_accuracy': 0.12370520085096359,
 'factorized_top_k/top_20_categorical_accuracy': 0.14446045458316803,
 'factorized_top_k/top_500_categorical_accuracy': 0.32791391015052795,
 'loss': 70140.78125,
 'regularization_loss': 0,
 'total_loss': 70140.78125}

In [ ]:
# Getting a list of 500 possible movies from the model. 

# Create a model that takes in raw query features, and
index_500 = tfrs.layers.factorized_top_k.BruteForce(model_1.query_model, k=500)
# recommends movies out of the entire movies dataset.
index_500.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(200).map(lambda x: x["candidate_title"]), movies.batch(200).map(model_1.candidate_model)))
)

# Get recommendations.

user_177165 = {
 'user_age_binned': tf.constant([2]),
 'user_gender': tf.constant([b'M']) ,
 'user_id': tf.constant([b'177165']),
 'user_occupation': tf.constant([b'college/grad student'])}


_, titles = index_500(user_177165)

print(f"Recommendations for user 177165: {titles[0, :3]}")


Recommendations for user 177165: [b'fast_+cheap++out+of+control+1997' b'unhook+the+stars+1996'
 b'london+to+brighton+2006']


In [ ]:
# Save the index (model).
tf.saved_model.save(index_500, '/content/gdrive/MyDrive/COMP585/model_retrieval')

# Ranking Model

## Data

In [ ]:
if 'google.colab' in str(get_ipython()):
  ratings = pd.read_csv('/content/gdrive/MyDrive/COMP585/data.csv', sep=',')
  movies = pd.read_csv('/content/gdrive/MyDrive/COMP585/data_movie.csv', sep=',')
  users = pd.read_csv('/content/gdrive/MyDrive/COMP585/data_user.csv', sep=';')
else:
  ratings = pd.read_csv('data.csv', sep=',')
  movies = pd.read_csv('data_movie.csv', sep=',')
  users = pd.read_csv('data_user.csv', sep=';')


## Preprocess

In [ ]:
# movies

# length binned:
bins = [0,30, 90, 120, 200,1500]
labels = [0,1,2,3,4]
movies['length_binned'] = pd.cut(movies['length'], bins=bins, labels=labels)

# year binned
movies['release_date'] = movies['release_date'].fillna('2000-01-01') # fill na !!!!
movies['original_language'] = movies['original_language'].fillna("en").astype(str) # fill na !!!!


movies['release_date'] = movies['release_date'].apply(lambda row:  datetime.strptime(str(row),'%Y-%m-%d').year ) 

bins = [1850,1920,1970,1990,2010,2020,2025]
labels = [0,1,2,3,4,5]
movies['year_binned'] = pd.cut(movies['release_date'], bins=bins, labels=labels)

movies['genres'] = movies.apply(lambda row : eval(row['genres']), axis = 1)
movies['genres'] = movies['genres'].apply(lambda row : [d['name'] for d in row if 'id' in d])

unique_genres = movies["genres"].explode().unique() 

movies = pd.concat([
        movies[['movieID','length_binned', 'year_binned', 'original_language']],
        movies.genres.apply(lambda x: pd.Series(1, x)).fillna(0)
    ], axis=1)

In [ ]:
movies

,movieID,length_binned,year_binned,original_language,Comedy,Mystery,Crime,Drama,Romance,Documentary,...,Adventure,War,Horror,Western,Fantasy,Family,History,TV Movie,Music,Foreign
0,++++++1959,2,1,el,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,++++2013,1,4,fa,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,+la+mode+1993,1,3,en,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,+laventure+2008,2,3,fr,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,+nos+amours+1983,2,2,fr,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25502,zulu+1964,3,1,en,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25503,zulu+2013,2,4,en,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25504,zulu+dawn+1979,2,2,en,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25505,zus++zo+2001,2,3,nl,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#users

bins = [0,18,25, 35, 45, 55, 65,100]
labels = [0,1,2,3,4,5,6]
users['age_binned'] = pd.cut(users['age'], bins=bins, labels=labels)

users = users[['userID', 'age','gender', 'age_binned', 'occupation']]
users['userID'] = users['userID'].astype(str)

users

,userID,age,gender,age_binned,occupation
0,1,34,M,2,sales/marketing
1,2,33,M,2,college/grad student
2,3,29,M,2,scientist
3,4,30,M,2,other or not specified
4,5,26,M,2,scientist
...,...,...,...,...,...
999995,999996,31,M,2,scientist
999996,999997,27,M,2,scientist
999997,999998,37,M,3,executive/managerial
999998,999999,29,M,2,scientist


In [ ]:
ratings['userID'] = ratings['userID'].astype(str)
ratings

,userID,movieID,ratings
0,340626,acapulco_+prima+spiaggia...+a+sinistra+1983,4
1,26447,otello+1986,3
2,172600,holes+2003,2
3,314533,my+brother+the+terrorist+2014,4
4,249289,sommer+der+gaukler+2011,2
...,...,...,...
199995,47231,the+zone+2007,4
199996,375329,shall+we+dance+1937,5
199997,63541,the+shawshank+redemption+1994,4
199998,255488,men+in+black+1997,5


In [ ]:
N = ratings.shape[0]
N_train = N*80//100
N_test = N - N_train

In [ ]:
N_train, N_test

(160000, 40000)

In [ ]:
ratings = ratings.merge(movies, left_on='movieID', right_on='movieID', how='left')
ratings = ratings.merge(users, left_on='userID', right_on='userID', how='left')

In [ ]:
ratings.isna().sum()

userID                  0
movieID                 0
ratings                 0
length_binned        2113
year_binned           229
original_language     229
Comedy                229
Mystery               229
Crime                 229
Drama                 229
Romance               229
Documentary           229
Thriller              229
Action                229
Animation             229
Science Fiction       229
Adventure             229
War                   229
Horror                229
Western               229
Fantasy               229
Family                229
History               229
TV Movie              229
Music                 229
Foreign               229
age                     0
gender                  0
age_binned              0
occupation              0
dtype: int64

In [ ]:
ratings['length_binned'] = ratings['length_binned'].fillna(2) ##########################
ratings['year_binned'] = ratings['year_binned'].fillna(2) ##########################
ratings = ratings.fillna(0)
ratings

,userID,movieID,ratings,length_binned,year_binned,original_language,Comedy,Mystery,Crime,Drama,...,Fantasy,Family,History,TV Movie,Music,Foreign,age,gender,age_binned,occupation
0,340626,acapulco_+prima+spiaggia...+a+sinistra+1983,4,2,2,it,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27,F,2,other or not specified
1,26447,otello+1986,3,2,2,en,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27,M,2,executive/managerial
2,172600,holes+2003,2,2,3,en,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,23,M,1,executive/managerial
3,314533,my+brother+the+terrorist+2014,4,2,4,en,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,34,M,2,college/grad student
4,249289,sommer+der+gaukler+2011,2,2,4,de,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30,M,2,scientist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,47231,the+zone+2007,4,2,3,es,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27,M,2,college/grad student
199996,375329,shall+we+dance+1937,5,2,1,en,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,33,M,2,academic/educator
199997,63541,the+shawshank+redemption+1994,4,3,3,en,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27,M,2,college/grad student
199998,255488,men+in+black+1997,5,2,3,en,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30,M,2,sales/marketing


In [ ]:
ratings.isna().sum()

userID               0
movieID              0
ratings              0
length_binned        0
year_binned          0
original_language    0
Comedy               0
Mystery              0
Crime                0
Drama                0
Romance              0
Documentary          0
Thriller             0
Action               0
Animation            0
Science Fiction      0
Adventure            0
War                  0
Horror               0
Western              0
Fantasy              0
Family               0
History              0
TV Movie             0
Music                0
Foreign              0
age                  0
gender               0
age_binned           0
occupation           0
dtype: int64

In [ ]:
ratings = ratings.drop_duplicates()
ratings

,userID,movieID,ratings,length_binned,year_binned,original_language,Comedy,Mystery,Crime,Drama,...,Fantasy,Family,History,TV Movie,Music,Foreign,age,gender,age_binned,occupation
0,340626,acapulco_+prima+spiaggia...+a+sinistra+1983,4,2,2,it,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27,F,2,other or not specified
1,26447,otello+1986,3,2,2,en,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27,M,2,executive/managerial
2,172600,holes+2003,2,2,3,en,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,23,M,1,executive/managerial
3,314533,my+brother+the+terrorist+2014,4,2,4,en,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,34,M,2,college/grad student
4,249289,sommer+der+gaukler+2011,2,2,4,de,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30,M,2,scientist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,47231,the+zone+2007,4,2,3,es,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27,M,2,college/grad student
199996,375329,shall+we+dance+1937,5,2,1,en,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,33,M,2,academic/educator
199997,63541,the+shawshank+redemption+1994,4,3,3,en,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,27,M,2,college/grad student
199998,255488,men+in+black+1997,5,2,3,en,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30,M,2,sales/marketing


In [ ]:
ratings['original_language'] = ratings['original_language'].replace(to_replace = 0, value = "en")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))

In [ ]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movieID"],
    "user_id": x["userID"],
    "user_rating": x["ratings"],
    "movie_language": x["original_language"],
    "user_occupation": x["occupation"],
    "movie_length_binned": x['length_binned'],	
    "movie_year_binned": x['year_binned'],
    "user_gender": x["gender"],
    "user_age_binned": x["age_binned"],
    "comedy" : x["Comedy"],
    "mystery": x["Mystery"],            
    "crime": x["Crime"],              
    "drama": x["Drama"],              
    "romance": x["Romance"],            
    "documentary": x["Documentary"],        
    "thriller": x["Thriller"],           
    "action": x["Action"],             
    "animation": x["Animation"],          
    "science_fiction": x["Science Fiction"],    
    "adventure": x["Adventure"],          
    "war": x["War"],               
    "horror": x["Horror"],             
    "western": x["Western"],            
    "fantasy": x["Fantasy"],            
    "family": x["Family"],             
    "history": x["History"],            
    "tv_movie": x["TV Movie"],           
    "music": x["Music"],              
    "foreign": x["Foreign"],    
})



In [ ]:
#tf.random.set_seed(42)
#shuffled = ratings.shuffle(N, seed=42, reshuffle_each_iteration=False)

#train = shuffled.take(N_train)
#test = shuffled.skip(N_train).take(N_test)

train = ratings.take(N_train)
test = ratings.skip(N_train).take(N_test)

cached_train = train.shuffle(N_train).batch(2048).cache()
cached_test = test.batch(2048).cache()

In [ ]:
movie_titles = ratings.batch(N).map(lambda x: x["movie_title"])
years_binned = ratings.batch(N).map(lambda x: x["movie_year_binned"])
lengths_binned = ratings.batch(N).map(lambda x: x["movie_length_binned"])
user_ids = ratings.batch(N).map(lambda x: x["user_id"])
user_genders = ratings.batch(N).map(lambda x: x["user_gender"])
user_ages_binned = ratings.batch(N).map(lambda x: x["user_age_binned"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_years_binned = np.unique(np.concatenate(list(years_binned)))
unique_lengths_binned = np.unique(np.concatenate(list(lengths_binned)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))
unique_user_genders = np.unique(np.concatenate(list(user_genders)))
unique_user_ages_binned = np.unique(np.concatenate(list(user_ages_binned)))

In [ ]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # users features
    self.age_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_user_ages_binned, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ages_binned) + 1, 8),
    ])
    
    self.gender_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_genders, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_genders) + 1, 8),
    ])

    self.occupation_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_occupations, mask_token=None),
        tf.keras.layers.Embedding(len(unique_occupations) + 1, 8),
    ])

    #movie features

    self.lenght_embedding = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(
      vocabulary=unique_lengths_binned, mask_token=None),
    tf.keras.layers.Embedding(len(unique_lengths_binned) + 1, 8)])

    self.year_embedding = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(
      vocabulary=unique_years_binned, mask_token=None),
    tf.keras.layers.Embedding(len(unique_years_binned) + 1, 8)])

    self.language_embedding = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_languages, mask_token=None),
    tf.keras.layers.Embedding(len(unique_languages) + 1, 8)])

    self.genre_embedding = tf.keras.layers.Dense(32)

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(32, activation="relu"),
      tf.keras.layers.Dense(32, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])
    


  def call(self, inputs):

    #user_id, movie_title = inputs

    Comedy =  tf.expand_dims(inputs["candidate_Comedy"], axis=1)
    Mystery =  tf.expand_dims(inputs["candidate_Mystery"], axis=1)
    Crime  =  tf.expand_dims(inputs["candidate_Crime"], axis=1)
    Drama =  tf.expand_dims(inputs["candidate_Drama"], axis=1)
    Romance  =  tf.expand_dims(inputs["candidate_Romance"], axis=1)
    Documentary =  tf.expand_dims(inputs["candidate_Documentary"], axis=1)
    Thriller  =  tf.expand_dims(inputs["candidate_Thriller"], axis=1)
    Action =  tf.expand_dims(inputs["candidate_Action"], axis=1)
    Animation =  tf.expand_dims(inputs["candidate_Animation"], axis=1)
    Science_Fiction =  tf.expand_dims(inputs["candidate_Science_Fiction"], axis=1)
    Adventure =  tf.expand_dims(inputs["candidate_Adventure"], axis=1)
    War  =  tf.expand_dims(inputs["candidate_War"], axis=1)
    Horror  =  tf.expand_dims(inputs["candidate_Horror"], axis=1)
    Western =  tf.expand_dims(inputs["candidate_Western"], axis=1)
    Fantasy =  tf.expand_dims(inputs["candidate_Fantasy"], axis=1)
    Family =  tf.expand_dims(inputs["candidate_Family"], axis=1)
    History =  tf.expand_dims(inputs["candidate_History"], axis=1)
    TV_Movie =  tf.expand_dims(inputs["candidate_TV_Movie"], axis=1)
    Music =  tf.expand_dims(inputs["candidate_Music"], axis=1)
    Foreign  =  tf.expand_dims(inputs["candidate_Foreign"], axis=1)

    genres = tf.concat(values=[Comedy,Mystery,Crime,Drama,Romance,Documentary,Thriller,Action,
                               Animation,Science_Fiction,Adventure,War,Horror,Western,
                               Fantasy,Family,History,TV_Movie,Music,Foreign],axis=1)


    user_embedding = self.user_embeddings(inputs['user_id'])
    movie_embedding = self.movie_embeddings(inputs['movie_title'])
    age_embedding = self.age_embedding(inputs['user_age_binned'])
    gender_embedding = self.gender_embedding(inputs['user_gender'])
    occupation_embedding = self.occupation_embedding(inputs['user_occupation'])
    lenght_embedding = self.lenght_embedding(inputs['movie_length_binned'])
    year_embedding = self.year_embedding(inputs['movie_year_binned'])
    genre_embedding = self.genre_embedding(genres)
    language_embedding = self.language_embedding(inputs['movie_language'])


    return self.ratings(tf.concat([user_embedding, movie_embedding,age_embedding,gender_embedding,occupation_embedding,lenght_embedding,year_embedding,genre_embedding,language_embedding], axis=1))


class RecRankModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    
    return self.ranking_model({
        "user_id": features["user_id"],
        "movie_title": features["movie_title"],
        "user_age_binned": features["user_age_binned"],
        "user_gender": features["user_gender"],
        "user_occupation": features["user_occupation"], 
        "movie_language": features['movie_language'],
        "movie_length_binned": features['movie_length_binned'],	
        "movie_year_binned": features['movie_year_binned'],
        "candidate_Comedy" : features["comedy"],
        "candidate_Mystery": features["mystery"],            
        "candidate_Crime": features["crime"],              
        "candidate_Drama": features["drama"],              
        "candidate_Romance": features["romance"],            
        "candidate_Documentary": features["documentary"],        
        "candidate_Thriller": features["thriller"],           
        "candidate_Action": features["action"],             
        "candidate_Animation": features["animation"],          
        "candidate_Science_Fiction": features["science_fiction"],    
        "candidate_Adventure": features["adventure"],          
        "candidate_War": features["war"],               
        "candidate_Horror": features["horror"],             
        "candidate_Western": features["western"],            
        "candidate_Fantasy": features["fantasy"],            
        "candidate_Family": features["family"],             
        "candidate_History": features["history"],            
        "candidate_TV_Movie": features["tv_movie"],           
        "candidate_Music": features["music"],              
        "candidate_Foreign": features["foreign"],
        })

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)


In [ ]:
model_2 = RecRankModel()
model_2.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
model_2.fit(cached_train, epochs=10, validation_data=cached_test)

Epoch 1/10
79/79 [==============================] - 16s 116ms/step - root_mean_squared_error: 2.4923 - loss: 6.0793 - regularization_loss: 0.0000e+00 - total_loss: 6.0793 - val_root_mean_squared_error: 0.7407 - val_loss: 0.5392 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.5392
Epoch 2/10
79/79 [==============================] - 1s 9ms/step - root_mean_squared_error: 0.7396 - loss: 0.5477 - regularization_loss: 0.0000e+00 - total_loss: 0.5477 - val_root_mean_squared_error: 0.7371 - val_loss: 0.5344 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.5344
Epoch 3/10
79/79 [==============================] - 1s 9ms/step - root_mean_squared_error: 0.7362 - loss: 0.5428 - regularization_loss: 0.0000e+00 - total_loss: 0.5428 - val_root_mean_squared_error: 0.7345 - val_loss: 0.5300 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.5300
Epoch 4/10
79/79 [==============================] - 1s 9ms/step - root_mean_squared_error: 0.7340 - loss: 0.5397 - regularization_los

In [ ]:
model_2.evaluate(cached_test, return_dict=True)

20/20 [==============================] - 0s 6ms/step - root_mean_squared_error: 0.7173 - loss: 0.5136 - regularization_loss: 0.0000e+00 - total_loss: 0.5136


{'root_mean_squared_error': 0.7173391580581665,
 'loss': 0.5002753734588623,
 'regularization_loss': 0,
 'total_loss': 0.5002753734588623}

In [ ]:
test_ratings = {}
test_movie_titles = ["zulu+dawn+1979", "the+little+mermaid+1989", "shrek+2+2004"]


for movie_title in test_movie_titles:
  test_ratings[movie_title] = model_2({
      "user_id": np.array(["177165"]),
      "movie_title": np.array([movie_title]),
      'user_age_binned': np.array([2]),
      'user_gender': np.array(["M"]),
      "user_occupation": np.array(["scientist"]) , 
      "movie_language": np.array(["en"]) ,
      "movie_length_binned":np.array([2]),
      "movie_year_binned":np.array([2]),
      "comedy" : np.array([1]),
      "mystery": np.array([1]),            
      "crime": np.array([1]),              
      "drama": np.array([1]),              
      "romance": np.array([1]),            
      "documentary": np.array([1]),        
      "thriller": np.array([1]),           
      "action": np.array([1]),             
      "animation": np.array([1]),          
      "science_fiction": np.array([1]),    
      "adventure": np.array([1]),          
      "war": np.array([1]),               
      "horror": np.array([1]),             
      "western": np.array([1]),            
      "fantasy": np.array([1]),            
      "family": np.array([1]),             
      "history": np.array([1]),            
      "tv_movie": np.array([1]),           
      "music": np.array([1]),              
      "foreign": np.array([1])

  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
the+little+mermaid+1989: [[3.6372592]]
shrek+2+2004: [[3.6267524]]
zulu+dawn+1979: [[3.601904]]


In [ ]:
# Save the index.
tf.saved_model.save(model_2, '/content/gdrive/MyDrive/COMP585/model_rank')

In [ ]:
model_2.save_weights("/content/gdrive/MyDrive/COMP585/weights_rank")

# Using the 2 steps (Retrieval -> Ranking)

In [ ]:
# Load it back; can also be done in TensorFlow Serving.
loaded_retrieval = tf.saved_model.load('/content/gdrive/MyDrive/COMP585/model_retrieval')

user_177165 = {
 'user_age_binned': tf.constant([2]),
 'user_gender': tf.constant([b'M']) ,
 'user_id': tf.constant([b'177165']),
 'user_occupation': tf.constant([b'college/grad student']) }


_, titles = loaded_retrieval(user_177165)

print(f"Recommendations for user 177165: {titles[0, :20]}")


Recommendations for user 177165: [b'fast_+cheap++out+of+control+1997' b'unhook+the+stars+1996'
 b'london+to+brighton+2006' b'room+237+2012' b'the+mask+1994'
 b'sands+of+iwo+jima+1949' b'shotgun+stories+2007'
 b'gurren+lagann+the+movie+the+lights+in+the+sky+are+stars+2009'
 b'whitey+united+states+of+america+v.+james+j.+bulger+2014'
 b'elysium+2013' b'toy+story+3+2010' b'sleepy+hollow+1999' b'stay+2005'
 b'the+ladykillers+1955' b'from+beyond+1986' b'the+first+grader+2010'
 b'panther+1995' b'running+scared+2006' b'the+square+2008'
 b'moonlight+mile+2002']


In [ ]:
loaded_rank =  RecRankModel()
loaded_rank.load_weights("/content/gdrive/MyDrive/COMP585/weights_rank")

In [ ]:
pred_ratings = np.zeros((500,))

for i in range(500):
  # Pass a user id in, movie id and perform predictions:
  pred_ratings[i] = loaded_rank({
      "user_id": np.array(["177165"]),
      "movie_title":[titles[0][i]],
      'user_age_binned': np.array([2]),
      'user_gender': np.array(["M"]),
      "user_occupation": np.array(["scientist"]) , 
      "movie_language": np.array(["en"]) ,
      "movie_length_binned":np.array([2]),
      "movie_year_binned":np.array([2]),
      "comedy" : np.array([1]),
      "mystery": np.array([1]),            
      "crime": np.array([1]),              
      "drama": np.array([1]),              
      "romance": np.array([1]),            
      "documentary": np.array([1]),        
      "thriller": np.array([1]),           
      "action": np.array([1]),             
      "animation": np.array([1]),          
      "science_fiction": np.array([1]),    
      "adventure": np.array([1]),          
      "war": np.array([1]),               
      "horror": np.array([1]),             
      "western": np.array([1]),            
      "fantasy": np.array([1]),            
      "family": np.array([1]),             
      "history": np.array([1]),            
      "tv_movie": np.array([1]),           
      "music": np.array([1]),              
      "foreign": np.array([1])
  }).numpy()[0][0]

order = np.argsort(-pred_ratings)[0:20]


pred_movies = titles[0].numpy()[list(order)]


In [ ]:
print(pred_movies)

[b'raiders+of+the+lost+ark+1981' b'the+shawshank+redemption+1994'
 b'up+2009' b'the+lord+of+the+rings+the+fellowship+of+the+ring+2001'
 b'the+dark+knight+2008' b'inception+2010' b'the+matrix+1999'
 b'whiplash+2014' b'the+lord+of+the+rings+the+return+of+the+king+2003'
 b'the+lord+of+the+rings+the+two+towers+2002' b'fight+club+1999'
 b'alien+1979' b'forrest+gump+1994' b'batman+begins+2005' b'walle+2008'
 b'reservoir+dogs+1992' b'the+lion+king+1994' b'toy+story+3+2010'
 b'the+dark+knight+rises+2012' b'shutter+island+2010']
